In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir('../input/'))

['min_and_max_value.csv', 'st_data_itemChargeFull.tsv']


In [4]:
item_df = pd.read_csv('../input/st_data_itemChargeFull.tsv', sep='\t')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#날짜 타입 변경
item_df.date = pd.to_datetime(item_df.date, format='%Y%m%d')

itemcode로 조회되지 않는 데이터가 있음 -> 해당 데이터의 itemcode는 int 타입으로 되어있음

In [6]:
#itemcode 타입 변경(str, int 혼재 -> str로)
item_df.itemcode = item_df.itemcode.astype(str)

데이터의 기간 확인 : 2015-01-07 ~ 2018-12-26 사이의 증시 데이터

In [7]:
item_df.date.min(), item_df.date.max()

(Timestamp('2015-01-07 00:00:00'), Timestamp('2018-12-26 00:00:00'))

## 1. 2016년 대비 2018년에 성장 폭이 큰 기업 추출

### 1. 거래 정지가 되지 않았던 기업만 추출

In [9]:
train_df = item_df.loc[item_df.trade_stop_yn == 'N']

### 2. 오기?? 로 판단되는 데이터 처리

* 주식 상한선인 30%가 넘는 수치가 올랐을 시점 찾기

In [22]:
sample = train_df.loc[train_df.itemcode == '18290']

In [84]:
tt = sample.close_val.rolling(window=2).sum()

In [90]:
len(sample)

312

In [49]:
sample['test'] = sample.close_val.rolling(window=2).sum()
sample['sub'] = sample['test'] - sample['close_val']*2
sample['test2'] = abs(sample['sub'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
for j,k in sample.iterrows():
    try:
        x1 = k.test2
        x2 = k.close_val * 0.5
        if x1 >= x2:
            print(k.itemname, '  ', j,' 번째')
            continue;
    except ValueError:
        print('last value')
        continue;

지엠피    201157  번째
지엠피    201190  번째
지엠피    201207  번째


In [82]:
sample

,itemcode,itemname,market,date,close_val,change_val,change_rate,acc_quant,debt_total,face_val,frgn_hold_ratio,high_val,listed_stock,low_val,market_sum,net_income,open_val,operating_profit,operating_profit_increasing_rate,prev_quant,property_total,reserve_ratio,roa,roe,sales,sales_increasing_rate,trade_stop_yn,test,sub,test2
201106,18290,지엠피,KOSDAQ,2015-01-07,558.0,1.0,0.18,274049.0,542.0,500,6.70,560.0,61397542,545.0,342.0,107.0,560.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,NaN,NaN,NaN
201107,18290,지엠피,KOSDAQ,2015-01-13,576.0,-11.0,-1.91,342095.0,542.0,500,6.76,594.0,61397542,571.0,353.0,107.0,590.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1134.0,-18.0,18.0
201108,18290,지엠피,KOSDAQ,2015-01-16,566.0,-9.0,-1.59,373931.0,542.0,500,6.79,580.0,61397542,550.0,347.0,107.0,580.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1142.0,10.0,10.0
201109,18290,지엠피,KOSDAQ,2015-01-19,571.0,5.0,0.88,163960.0,542.0,500,6.80,573.0,61397542,560.0,350.0,107.0,569.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1137.0,-5.0,5.0
201110,18290,지엠피,KOSDAQ,2015-01-22,588.0,7.0,1.19,1878914.0,542.0,500,6.76,619.0,61397542,570.0,361.0,107.0,581.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1159.0,-17.0,17.0
201111,18290,지엠피,KOSDAQ,2015-01-28,574.0,-7.0,-1.22,361408.0,542.0,500,6.91,586.0,61397542,573.0,352.0,107.0,586.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1162.0,14.0,14.0
201112,18290,지엠피,KOSDAQ,2015-02-03,571.0,-1.0,-0.18,410295.0,542.0,500,6.85,579.0,61397542,564.0,350.0,107.0,572.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1145.0,3.0,3.0
201113,18290,지엠피,KOSDAQ,2015-02-06,590.0,15.0,2.54,2739819.0,542.0,500,6.90,611.0,61397542,573.0,362.0,107.0,577.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1161.0,-19.0,19.0
201114,18290,지엠피,KOSDAQ,2015-02-09,587.0,-3.0,-0.51,1472628.0,542.0,500,6.87,614.0,61397542,585.0,360.0,107.0,595.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1177.0,3.0,3.0
201115,18290,지엠피,KOSDAQ,2015-02-12,595.0,17.0,2.86,761491.0,542.0,500,6.84,597.0,61397542,572.0,365.0,107.0,578.0,134.0,300.71,328341.0,1240.0,326.88,10.81,17.62,1095.0,56.7,N,1182.0,-8.0,8.0


In [ ]:
train_df = train_df.loc[train_df.date >= '2016-01-01']

In [14]:
#종목코드 추출
code_list = train_df.itemcode.unique()

In [ ]:
s_df = pd.DataFrame({'id': code_list, 'min':0, 'max':0})
for i in code_list:
    sample = train_df.loc[train_df.itemcode ==i]
    s_df.loc[s_df.id == i, 'min'] = sample.close_val[:10].mean()
    s_df.loc[s_df.id == i, 'max'] = sample.close_val[-10:].mean()
    
s_df['diff'] = s_df['max'] - s_df['min']
s_df['ratio'] = s_df['diff']/s_df['min']

s_df.to_csv('min_and_max_value.csv', index=False)

In [ ]:
s_df = pd.read_csv('min_and_max_value.csv')

In [ ]:
s_df.head()

In [ ]:
s_df.sort_values('ratio', ascending=False).head(10)

In [ ]:
for i in s_df.sort_values('ratio', ascending=False).head(10)['id']:
    print(train_df.loc[train_df.itemcode == i, 'itemname'].unique())

In [ ]:
train_df.loc[train_df.itemname == '제이준코스메틱']

### 개별 항목 조회

* '동부건설우'의 가격이 이상하다?

In [ ]:
item_df.loc[item_df.itemcode == '18290', 'itemname'].unique()

In [ ]:
sample = item_df.loc[item_df.itemcode == '000020']

In [ ]:
plt.plot(sample.date, sample.close_val)

In [ ]:
#len(item_df.itemcode.unique())
item_df[['itemcode','close_val']].groupby('itemcode').describe()